In [1]:
#imports
import pandas as pd

In [2]:
#read in the raw csv file
crime_data = pd.read_csv("../data/BPD_crime_data.csv", encoding= 'utf-8')
crime_data.head(2) #check to ensure it worked
crime_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 313634 entries, 0 to 313633
Data columns (total 16 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   CrimeDate        313634 non-null  object 
 1   CrimeTime        313609 non-null  object 
 2   CrimeCode        313634 non-null  object 
 3   Location         312020 non-null  object 
 4   Description      313634 non-null  object 
 5   Inside/Outside   275344 non-null  object 
 6   Weapon           66908 non-null   object 
 7   Post             312916 non-null  float64
 8   District         312916 non-null  object 
 9   Neighborhood     312899 non-null  object 
 10  Longitude        313404 non-null  float64
 11  Latitude         313404 non-null  float64
 12  Location 1       0 non-null       float64
 13  Premise          275094 non-null  object 
 14  vri_name1        37056 non-null   object 
 15  Total Incidents  313634 non-null  int64  
dtypes: float64(4), int64(1), object(11)
me

In [3]:
#Next we will begin the cleaning process

#Let's look to see what columns we can get rid of

#set the column names to be camelCase convention, contert the Inside/Outside column name
#simplest one-liner to just replace the / with ""
crime_data_correct_columns = crime_data.copy()
crime_data_correct_columns = crime_data_correct_columns.rename(columns= lambda x: x.replace("/", ""));
crime_data_correct_columns = crime_data_correct_columns.rename(columns= lambda x: x.replace(" ", ""));

#First, we notice that the 'Total Incidents' column is always 1, which tells us nothing so we get rid of it
#proven here by seeing the count is equal to the number of rows
print("Number of 1's in total incidents column: ", crime_data_correct_columns[crime_data_correct_columns.TotalIncidents == 1].count())

#Let's see if the info on the data:
crime_data.info()

#We notice that the 'Location 1' column has 0 non-null values, so that column can also be dropped
#It looks like the contents of the vri_name1 column are contained in the District column.  That, combined with the
#fact that there are only 37,056 non-null values in the vri_name1 column leads me to the decision that it can be
#dropped for simplicity.  This should be noted that our results will not have any analysis of the vri_name1 column
#(some are slightly different than the district and this content will not be represented, we deem it unimportant 
#for the time being but it could come into play later. 


#Drop those 3 columns
crime_data_correct_columns = crime_data_correct_columns.drop(columns=['TotalIncidents', 'vri_name1', 'Location1'])

crime_data_correct_columns.head(2)
crime_data_correct_columns.info()

Number of 1's in total incidents column:  CrimeDate         313634
CrimeTime         313609
CrimeCode         313634
Location          312020
Description       313634
InsideOutside     275344
Weapon             66908
Post              312916
District          312916
Neighborhood      312899
Longitude         313404
Latitude          313404
Location1              0
Premise           275094
vri_name1          37056
TotalIncidents    313634
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 313634 entries, 0 to 313633
Data columns (total 16 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   CrimeDate        313634 non-null  object 
 1   CrimeTime        313609 non-null  object 
 2   CrimeCode        313634 non-null  object 
 3   Location         312020 non-null  object 
 4   Description      313634 non-null  object 
 5   Inside/Outside   275344 non-null  object 
 6   Weapon           66908 non-null   object 
 7   Post  

In [4]:
#Now that we have the columns we want, lets deal with the NaN values
#Judging by the output of info on the data frame:
#there are 313,634 entries.  The columns of CrimeTime, Location, InsideOutside, Weapon, Post, District,
#Neighborhood, Longitude, Latitude, Premise all contain null values.

#There are only 66,908 non-null values for Weapon, so I'm going to assume that if weapon is NaN, then there was
#no weapon used, as I believe that is more likely than these entries all being "missing" or "incomplete"

#copy the df to a new df to modify
crime_data_cleaned = crime_data_correct_columns.copy()

#build off the correct columns df and fill them with "None"
crime_data_cleaned.Weapon.fillna("None", inplace=True)

#continuing, for the CrimeTime, there are 25 NaN values.  Since there are little of them compared to the size
#of the dataset and to avoid adding any bias by filling with a time, I will remove these NaN entries
crime_data_cleaned.dropna(subset = ["CrimeTime"], inplace=True)

#same with Post, rather than filling and potentially introducing bias, we will remove rows with NaN there
crime_data_cleaned.dropna(subset = ["Post"], inplace=True)

#The rest of the columns that contain null values are String attributes and replacing the NaN values with "Unknown"
#we can do and take note of it so that we know about it when exploring the data.  But, it makes sense to keep
#these rows as we do not want to miss any information that these rows may contain.
crime_data_cleaned.Location.fillna("UNKNOWN", inplace= True)
crime_data_cleaned.InsideOutside.fillna("UNKNOWN", inplace= True)
crime_data_cleaned.Neighborhood.fillna("UNKNOWN", inplace= True)
crime_data_cleaned.Premise.fillna("UNKNOWN", inplace= True)

#We notice also that some values in the InsideOutside column are inconsistent - some labeled inside have values of
#just "I" and others have "Inside" normoalize these to be "I", "O", or the "Unknown"
crime_data_cleaned.loc[crime_data_cleaned.InsideOutside == "I", "InsideOutside"] = "Inside"
crime_data_cleaned.loc[crime_data_cleaned.InsideOutside == "O", "InsideOutside"] = "Outside"

#Again, there is an inconsistency in the weapon data - a few have FIRE instead of FIREARM, make the correction:
crime_data_cleaned.loc[crime_data_cleaned.Weapon == "FIRE", "Weapon"] = "FIREARM"

crime_data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 312891 entries, 0 to 313633
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   CrimeDate      312891 non-null  object 
 1   CrimeTime      312891 non-null  object 
 2   CrimeCode      312891 non-null  object 
 3   Location       312891 non-null  object 
 4   Description    312891 non-null  object 
 5   InsideOutside  312891 non-null  object 
 6   Weapon         312891 non-null  object 
 7   Post           312891 non-null  float64
 8   District       312891 non-null  object 
 9   Neighborhood   312891 non-null  object 
 10  Longitude      312891 non-null  float64
 11  Latitude       312891 non-null  float64
 12  Premise        312891 non-null  object 
dtypes: float64(3), object(10)
memory usage: 33.4+ MB


In [5]:
#Now that we have cleaned the data, let's write it back to a csv so it may be loaded from any notebook
crime_data_cleaned.to_csv('../data/cleaned_data.csv', index = False)